In [22]:
import pandas as pd
import numpy as np
from itertools import combinations
from scipy.stats import kruskal, mannwhitneyu


try:
    import scikit_posthocs as sp
    HAS_SCPH = True
except Exception:
    HAS_SCPH = False


def holm_adjust(pvals):

    m = len(pvals)
    order = np.argsort(pvals)
    adjusted = np.empty(m, dtype=float)
    prev = 0.0
    for rank, idx in enumerate(order, start=1):
        adj = (m - rank + 1) * pvals[idx]
        adj = min(1.0, max(adj, prev))
        adjusted[idx] = adj
        prev = adj
    return adjusted

def pairwise_matrix_from_pvals(labels, pair_pvals):

    mat = pd.DataFrame(np.nan, index=labels, columns=labels, dtype=float)
    for (a, b), p in pair_pvals.items():
        mat.loc[a, b] = p
        mat.loc[b, a] = p
    return mat


construct_prefixes = ["quality", "taste", "listen"]
trait_suffixes = [
    "openness",
    "conscientiousness",
    "extroversion",
    "agreeableness",
    "neuroticism",
    "featurebased",
]



traits_of_interest = ["openness"]


for trait_of_interest in traits_of_interest:

    analysis_df[f"{trait_of_interest}_bin"] = pd.cut(
        analysis_df[trait_of_interest],
        bins=[1.0, 3.6, 5.0],
        labels=["Low", "High"],
        include_lowest=True
    )


    all_results = {}

    for construct_prefix in construct_prefixes:

        constructs_columns = [f"{construct_prefix}_{suffix}" for suffix in trait_suffixes]

        mean_rows = []
        posthoc_results = {}

        levels = [lvl for lvl in analysis_df[f"{trait_of_interest}_bin"].dropna().unique()]

        for level in levels:
            row = {"Trait Level": level}
            subset = analysis_df[analysis_df[f"{trait_of_interest}_bin"] == level]


            for col in constructs_columns:
                if col in analysis_df.columns:
                    row[col] = round(subset[col].mean(), 2)
                else:
                    row[col] = np.nan


            values_per_construct = [subset[col].dropna() for col in constructs_columns if col in analysis_df.columns]
            if sum(len(v) > 0 for v in values_per_construct) >= 2:
                stat, p = kruskal(*values_per_construct)
                row["Kruskal Statistic"] = round(float(stat), 3)
                row["p-value"] = round(float(p), 3)
            else:
                row["Kruskal Statistic"] = np.nan
                row["p-value"] = np.nan


            if HAS_SCPH:
                long_df = pd.DataFrame({
                    "value": pd.concat(values_per_construct, ignore_index=True),
                    "group": np.concatenate([
                        np.repeat(lbl, len(values_per_construct[i]))
                        for i, lbl in enumerate([c for c in constructs_columns if c in analysis_df.columns])
                    ])
                })

                long_df = long_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["value"])
                if long_df["group"].nunique() >= 2 and len(long_df) > 0:
                    dunn = sp.posthoc_dunn(long_df, val_col="value", group_col="group", p_adjust="holm")

                    dunn = dunn.reindex(index=constructs_columns, columns=constructs_columns)
                    posthoc_results[level] = dunn
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)
            else:

                valid_constructs = [c for c in constructs_columns if c in analysis_df.columns]
                pairs = list(combinations(valid_constructs, 2))
                raw_p = []
                valid_pairs = []
                for a, b in pairs:
                    a_vals = subset[a].dropna()
                    b_vals = subset[b].dropna()
                    if len(a_vals) > 0 and len(b_vals) > 0:

                        u_stat, pval = mannwhitneyu(a_vals, b_vals, alternative="two-sided")
                        raw_p.append(pval)
                        valid_pairs.append((a, b))
                if raw_p:
                    adj = holm_adjust(np.array(raw_p, dtype=float))
                    pair_to_adj = {pair: p for pair, p in zip(valid_pairs, adj)}
                    mat = pairwise_matrix_from_pvals(constructs_columns, pair_to_adj)
                    posthoc_results[level] = mat
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)

            mean_rows.append(row)


        mean_table_df = pd.DataFrame(mean_rows)
        final_columns = ["Trait Level"] + constructs_columns + ["Kruskal Statistic", "p-value"]
        mean_table_df = mean_table_df[final_columns]


        all_results[construct_prefix] = {
            "mean_table": mean_table_df,
            "posthoc_results": posthoc_results
        }


    print(f"\n{'='*80}")
    print(f"ANALYSIS BY TRAIT: {trait_of_interest.upper()}")
    print(f"{'='*80}\n")

    for construct_prefix in construct_prefixes:
        print(f"\n{'='*80}")
        print(f"RESULTS FOR: {construct_prefix.upper()} for {trait_of_interest}")
        print(f"{'='*80}\n")

        print(f"Mean Table for {construct_prefix}:")
        print(all_results[construct_prefix]["mean_table"])
        print("\n")

        print(f"Post-hoc Results for {construct_prefix} (High Level Only):")
        if "High" in all_results[construct_prefix]["posthoc_results"]:
            print(all_results[construct_prefix]["posthoc_results"]["High"])
        else:
            print("No High level data available")
        print("\n")




ANALYSIS BY TRAIT: OPENNESS


RESULTS FOR: QUALITY for openness

Mean Table for quality:
  Trait Level  quality_openness  quality_conscientiousness  \
0         Low              4.20                       4.07   
1        High              4.24                       4.17   

   quality_extroversion  quality_agreeableness  quality_neuroticism  \
0                  4.27                   4.22                 4.28   
1                  4.34                   4.39                 4.35   

   quality_featurebased  Kruskal Statistic  p-value  
0                  4.24              7.124    0.212  
1                  4.40             15.783    0.007  


Post-hoc Results for quality (High Level Only):
                           quality_openness  quality_conscientiousness  \
quality_openness                        NaN                   1.000000   
quality_conscientiousness          1.000000                        NaN   
quality_extroversion               1.000000                   0.175792   
q

In [21]:
traits_of_interest = ["conscientiousness"]

for trait_of_interest in traits_of_interest:


    analysis_df[f"{trait_of_interest}_bin"] = pd.cut(
        analysis_df[trait_of_interest],
        bins=[1.0, 3.6, 5.0],
        labels=["Low", "High"],
        include_lowest=True
    )


    all_results = {}

    for construct_prefix in construct_prefixes:

        constructs_columns = [f"{construct_prefix}_{suffix}" for suffix in trait_suffixes]

        mean_rows = []
        posthoc_results = {}
        levels = [lvl for lvl in analysis_df[f"{trait_of_interest}_bin"].dropna().unique()]

        for level in levels:
            row = {"Trait Level": level}
            subset = analysis_df[analysis_df[f"{trait_of_interest}_bin"] == level]


            for col in constructs_columns:
                if col in analysis_df.columns:
                    row[col] = round(subset[col].mean(), 2)
                else:
                    row[col] = np.nan


            values_per_construct = [subset[col].dropna() for col in constructs_columns if col in analysis_df.columns]
            if sum(len(v) > 0 for v in values_per_construct) >= 2:
                stat, p = kruskal(*values_per_construct)
                row["Kruskal Statistic"] = round(float(stat), 3)
                row["p-value"] = round(float(p), 3)
            else:
                row["Kruskal Statistic"] = np.nan
                row["p-value"] = np.nan


            if HAS_SCPH:
                long_df = pd.DataFrame({
                    "value": pd.concat(values_per_construct, ignore_index=True),
                    "group": np.concatenate([
                        np.repeat(lbl, len(values_per_construct[i]))
                        for i, lbl in enumerate([c for c in constructs_columns if c in analysis_df.columns])
                    ])
                })

                long_df = long_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["value"])
                if long_df["group"].nunique() >= 2 and len(long_df) > 0:
                    dunn = sp.posthoc_dunn(long_df, val_col="value", group_col="group", p_adjust="holm")

                    dunn = dunn.reindex(index=constructs_columns, columns=constructs_columns)
                    posthoc_results[level] = dunn
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)
            else:

                valid_constructs = [c for c in constructs_columns if c in analysis_df.columns]
                pairs = list(combinations(valid_constructs, 2))
                raw_p = []
                valid_pairs = []
                for a, b in pairs:
                    a_vals = subset[a].dropna()
                    b_vals = subset[b].dropna()
                    if len(a_vals) > 0 and len(b_vals) > 0:

                        u_stat, pval = mannwhitneyu(a_vals, b_vals, alternative="two-sided")
                        raw_p.append(pval)
                        valid_pairs.append((a, b))
                if raw_p:
                    adj = holm_adjust(np.array(raw_p, dtype=float))
                    pair_to_adj = {pair: p for pair, p in zip(valid_pairs, adj)}
                    mat = pairwise_matrix_from_pvals(constructs_columns, pair_to_adj)
                    posthoc_results[level] = mat
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)

            mean_rows.append(row)


        mean_table_df = pd.DataFrame(mean_rows)
        final_columns = ["Trait Level"] + constructs_columns + ["Kruskal Statistic", "p-value"]
        mean_table_df = mean_table_df[final_columns]


        all_results[construct_prefix] = {
            "mean_table": mean_table_df,
            "posthoc_results": posthoc_results
        }

    print(f"\n{'='*80}")
    print(f"ANALYSIS BY TRAIT: {trait_of_interest.upper()}")
    print(f"{'='*80}\n")

    for construct_prefix in construct_prefixes:
        print(f"\n{'='*80}")
        print(f"RESULTS FOR: {construct_prefix.upper()} for {trait_of_interest}")
        print(f"{'='*80}\n")

        print(f"Mean Table for {construct_prefix}:")
        print(all_results[construct_prefix]["mean_table"])
        print("\n")

        print(f"Post-hoc Results for {construct_prefix} (High Level Only):")
        if "High" in all_results[construct_prefix]["posthoc_results"]:
            print(all_results[construct_prefix]["posthoc_results"]["High"])
        else:
            print("No High level data available")
        print("\n")




ANALYSIS BY TRAIT: CONSCIENTIOUSNESS


RESULTS FOR: QUALITY for conscientiousness

Mean Table for quality:
  Trait Level  quality_openness  quality_conscientiousness  \
0        High              4.23                       4.19   
1         Low              4.23                       4.05   

   quality_extroversion  quality_agreeableness  quality_neuroticism  \
0                  4.33                   4.38                 4.37   
1                  4.29                   4.28                 4.25   

   quality_featurebased  Kruskal Statistic  p-value  
0                  4.41             16.057    0.007  
1                  4.25              8.553    0.128  


Post-hoc Results for quality (High Level Only):
                           quality_openness  quality_conscientiousness  \
quality_openness                        NaN                   1.000000   
quality_conscientiousness          1.000000                        NaN   
quality_extroversion               0.944125              

In [23]:
traits_of_interest = ["extroversion"]

for trait_of_interest in traits_of_interest:


    analysis_df[f"{trait_of_interest}_bin"] = pd.cut(
        analysis_df[trait_of_interest],
        bins=[1.0, 3.6, 5.0],
        labels=["Low", "High"],
        include_lowest=True
    )


    all_results = {}

    for construct_prefix in construct_prefixes:

        constructs_columns = [f"{construct_prefix}_{suffix}" for suffix in trait_suffixes]

        mean_rows = []
        posthoc_results = {}
        levels = [lvl for lvl in analysis_df[f"{trait_of_interest}_bin"].dropna().unique()]

        for level in levels:
            row = {"Trait Level": level}
            subset = analysis_df[analysis_df[f"{trait_of_interest}_bin"] == level]


            for col in constructs_columns:
                if col in analysis_df.columns:
                    row[col] = round(subset[col].mean(), 2)
                else:
                    row[col] = np.nan


            values_per_construct = [subset[col].dropna() for col in constructs_columns if col in analysis_df.columns]
            if sum(len(v) > 0 for v in values_per_construct) >= 2:
                stat, p = kruskal(*values_per_construct)
                row["Kruskal Statistic"] = round(float(stat), 3)
                row["p-value"] = round(float(p), 3)
            else:
                row["Kruskal Statistic"] = np.nan
                row["p-value"] = np.nan


            if HAS_SCPH:
                long_df = pd.DataFrame({
                    "value": pd.concat(values_per_construct, ignore_index=True),
                    "group": np.concatenate([
                        np.repeat(lbl, len(values_per_construct[i]))
                        for i, lbl in enumerate([c for c in constructs_columns if c in analysis_df.columns])
                    ])
                })

                long_df = long_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["value"])
                if long_df["group"].nunique() >= 2 and len(long_df) > 0:
                    dunn = sp.posthoc_dunn(long_df, val_col="value", group_col="group", p_adjust="holm")

                    dunn = dunn.reindex(index=constructs_columns, columns=constructs_columns)
                    posthoc_results[level] = dunn
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)
            else:

                valid_constructs = [c for c in constructs_columns if c in analysis_df.columns]
                pairs = list(combinations(valid_constructs, 2))
                raw_p = []
                valid_pairs = []
                for a, b in pairs:
                    a_vals = subset[a].dropna()
                    b_vals = subset[b].dropna()
                    if len(a_vals) > 0 and len(b_vals) > 0:

                        u_stat, pval = mannwhitneyu(a_vals, b_vals, alternative="two-sided")
                        raw_p.append(pval)
                        valid_pairs.append((a, b))
                if raw_p:
                    adj = holm_adjust(np.array(raw_p, dtype=float))
                    pair_to_adj = {pair: p for pair, p in zip(valid_pairs, adj)}
                    mat = pairwise_matrix_from_pvals(constructs_columns, pair_to_adj)
                    posthoc_results[level] = mat
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)

            mean_rows.append(row)


        mean_table_df = pd.DataFrame(mean_rows)
        final_columns = ["Trait Level"] + constructs_columns + ["Kruskal Statistic", "p-value"]
        mean_table_df = mean_table_df[final_columns]


        all_results[construct_prefix] = {
            "mean_table": mean_table_df,
            "posthoc_results": posthoc_results
        }

    print(f"\n{'='*80}")
    print(f"ANALYSIS BY TRAIT: {trait_of_interest.upper()}")
    print(f"{'='*80}\n")

    for construct_prefix in construct_prefixes:
        print(f"\n{'='*80}")
        print(f"RESULTS FOR: {construct_prefix.upper()} for {trait_of_interest}")
        print(f"{'='*80}\n")

        print(f"Mean Table for {construct_prefix}:")
        print(all_results[construct_prefix]["mean_table"])
        print("\n")

        print(f"Post-hoc Results for {construct_prefix} (High Level Only):")
        if "High" in all_results[construct_prefix]["posthoc_results"]:
            print(all_results[construct_prefix]["posthoc_results"]["High"])
        else:
            print("No High level data available")
        print("\n")




ANALYSIS BY TRAIT: EXTROVERSION


RESULTS FOR: QUALITY for extroversion

Mean Table for quality:
  Trait Level  quality_openness  quality_conscientiousness  \
0         Low              4.31                       4.15   
1        High              4.06                       4.12   

   quality_extroversion  quality_agreeableness  quality_neuroticism  \
0                  4.33                   4.35                 4.34   
1                  4.29                   4.31                 4.30   

   quality_featurebased  Kruskal Statistic  p-value  
0                  4.35             15.160    0.010  
1                  4.37             11.977    0.035  


Post-hoc Results for quality (High Level Only):
                           quality_openness  quality_conscientiousness  \
quality_openness                        NaN                   1.000000   
quality_conscientiousness          1.000000                        NaN   
quality_extroversion               0.490078                   1.000

In [24]:
traits_of_interest = ["agreeableness"]

for trait_of_interest in traits_of_interest:


    analysis_df[f"{trait_of_interest}_bin"] = pd.cut(
        analysis_df[trait_of_interest],
        bins=[1.0, 3.6, 5.0],
        labels=["Low", "High"],
        include_lowest=True
    )


    all_results = {}

    for construct_prefix in construct_prefixes:

        constructs_columns = [f"{construct_prefix}_{suffix}" for suffix in trait_suffixes]

        mean_rows = []
        posthoc_results = {}
        levels = [lvl for lvl in analysis_df[f"{trait_of_interest}_bin"].dropna().unique()]

        for level in levels:
            row = {"Trait Level": level}
            subset = analysis_df[analysis_df[f"{trait_of_interest}_bin"] == level]


            for col in constructs_columns:
                if col in analysis_df.columns:
                    row[col] = round(subset[col].mean(), 2)
                else:
                    row[col] = np.nan


            values_per_construct = [subset[col].dropna() for col in constructs_columns if col in analysis_df.columns]
            if sum(len(v) > 0 for v in values_per_construct) >= 2:
                stat, p = kruskal(*values_per_construct)
                row["Kruskal Statistic"] = round(float(stat), 3)
                row["p-value"] = round(float(p), 3)
            else:
                row["Kruskal Statistic"] = np.nan
                row["p-value"] = np.nan


            if HAS_SCPH:
                long_df = pd.DataFrame({
                    "value": pd.concat(values_per_construct, ignore_index=True),
                    "group": np.concatenate([
                        np.repeat(lbl, len(values_per_construct[i]))
                        for i, lbl in enumerate([c for c in constructs_columns if c in analysis_df.columns])
                    ])
                })

                long_df = long_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["value"])
                if long_df["group"].nunique() >= 2 and len(long_df) > 0:
                    dunn = sp.posthoc_dunn(long_df, val_col="value", group_col="group", p_adjust="holm")

                    dunn = dunn.reindex(index=constructs_columns, columns=constructs_columns)
                    posthoc_results[level] = dunn
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)
            else:

                valid_constructs = [c for c in constructs_columns if c in analysis_df.columns]
                pairs = list(combinations(valid_constructs, 2))
                raw_p = []
                valid_pairs = []
                for a, b in pairs:
                    a_vals = subset[a].dropna()
                    b_vals = subset[b].dropna()
                    if len(a_vals) > 0 and len(b_vals) > 0:

                        u_stat, pval = mannwhitneyu(a_vals, b_vals, alternative="two-sided")
                        raw_p.append(pval)
                        valid_pairs.append((a, b))
                if raw_p:
                    adj = holm_adjust(np.array(raw_p, dtype=float))
                    pair_to_adj = {pair: p for pair, p in zip(valid_pairs, adj)}
                    mat = pairwise_matrix_from_pvals(constructs_columns, pair_to_adj)
                    posthoc_results[level] = mat
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)

            mean_rows.append(row)


        mean_table_df = pd.DataFrame(mean_rows)
        final_columns = ["Trait Level"] + constructs_columns + ["Kruskal Statistic", "p-value"]
        mean_table_df = mean_table_df[final_columns]


        all_results[construct_prefix] = {
            "mean_table": mean_table_df,
            "posthoc_results": posthoc_results
        }

    print(f"\n{'='*80}")
    print(f"ANALYSIS BY TRAIT: {trait_of_interest.upper()}")
    print(f"{'='*80}\n")

    for construct_prefix in construct_prefixes:
        print(f"\n{'='*80}")
        print(f"RESULTS FOR: {construct_prefix.upper()} for {trait_of_interest}")
        print(f"{'='*80}\n")

        print(f"Mean Table for {construct_prefix}:")
        print(all_results[construct_prefix]["mean_table"])
        print("\n")

        print(f"Post-hoc Results for {construct_prefix} (High Level Only):")
        if "High" in all_results[construct_prefix]["posthoc_results"]:
            print(all_results[construct_prefix]["posthoc_results"]["High"])
        else:
            print("No High level data available")
        print("\n")




ANALYSIS BY TRAIT: AGREEABLENESS


RESULTS FOR: QUALITY for agreeableness

Mean Table for quality:
  Trait Level  quality_openness  quality_conscientiousness  \
0        High              4.25                       4.16   
1         Low              4.00                       3.92   

   quality_extroversion  quality_agreeableness  quality_neuroticism  \
0                  4.35                   4.37                 4.35   
1                  3.92                   3.96                 4.08   

   quality_featurebased  Kruskal Statistic  p-value  
0                  4.36             20.922    0.001  
1                  4.29              4.621    0.464  


Post-hoc Results for quality (High Level Only):
                           quality_openness  quality_conscientiousness  \
quality_openness                        NaN                   0.797957   
quality_conscientiousness          0.797957                        NaN   
quality_extroversion               0.797957                   0.0

In [25]:
traits_of_interest = ["neuroticism"]

for trait_of_interest in traits_of_interest:


    analysis_df[f"{trait_of_interest}_bin"] = pd.cut(
        analysis_df[trait_of_interest],
        bins=[1.0, 3.6, 5.0],
        labels=["Low", "High"],
        include_lowest=True
    )


    all_results = {}

    for construct_prefix in construct_prefixes:

        constructs_columns = [f"{construct_prefix}_{suffix}" for suffix in trait_suffixes]

        mean_rows = []
        posthoc_results = {}
        levels = [lvl for lvl in analysis_df[f"{trait_of_interest}_bin"].dropna().unique()]

        for level in levels:
            row = {"Trait Level": level}
            subset = analysis_df[analysis_df[f"{trait_of_interest}_bin"] == level]


            for col in constructs_columns:
                if col in analysis_df.columns:
                    row[col] = round(subset[col].mean(), 2)
                else:
                    row[col] = np.nan


            values_per_construct = [subset[col].dropna() for col in constructs_columns if col in analysis_df.columns]
            if sum(len(v) > 0 for v in values_per_construct) >= 2:
                stat, p = kruskal(*values_per_construct)
                row["Kruskal Statistic"] = round(float(stat), 3)
                row["p-value"] = round(float(p), 3)
            else:
                row["Kruskal Statistic"] = np.nan
                row["p-value"] = np.nan


            if HAS_SCPH:
                long_df = pd.DataFrame({
                    "value": pd.concat(values_per_construct, ignore_index=True),
                    "group": np.concatenate([
                        np.repeat(lbl, len(values_per_construct[i]))
                        for i, lbl in enumerate([c for c in constructs_columns if c in analysis_df.columns])
                    ])
                })

                long_df = long_df.replace([np.inf, -np.inf], np.nan).dropna(subset=["value"])
                if long_df["group"].nunique() >= 2 and len(long_df) > 0:
                    dunn = sp.posthoc_dunn(long_df, val_col="value", group_col="group", p_adjust="holm")

                    dunn = dunn.reindex(index=constructs_columns, columns=constructs_columns)
                    posthoc_results[level] = dunn
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)
            else:

                valid_constructs = [c for c in constructs_columns if c in analysis_df.columns]
                pairs = list(combinations(valid_constructs, 2))
                raw_p = []
                valid_pairs = []
                for a, b in pairs:
                    a_vals = subset[a].dropna()
                    b_vals = subset[b].dropna()
                    if len(a_vals) > 0 and len(b_vals) > 0:

                        u_stat, pval = mannwhitneyu(a_vals, b_vals, alternative="two-sided")
                        raw_p.append(pval)
                        valid_pairs.append((a, b))
                if raw_p:
                    adj = holm_adjust(np.array(raw_p, dtype=float))
                    pair_to_adj = {pair: p for pair, p in zip(valid_pairs, adj)}
                    mat = pairwise_matrix_from_pvals(constructs_columns, pair_to_adj)
                    posthoc_results[level] = mat
                else:
                    posthoc_results[level] = pd.DataFrame(index=constructs_columns, columns=constructs_columns, dtype=float)

            mean_rows.append(row)


        mean_table_df = pd.DataFrame(mean_rows)
        final_columns = ["Trait Level"] + constructs_columns + ["Kruskal Statistic", "p-value"]
        mean_table_df = mean_table_df[final_columns]


        all_results[construct_prefix] = {
            "mean_table": mean_table_df,
            "posthoc_results": posthoc_results
        }

    print(f"\n{'='*80}")
    print(f"ANALYSIS BY TRAIT: {trait_of_interest.upper()}")
    print(f"{'='*80}\n")

    for construct_prefix in construct_prefixes:
        print(f"\n{'='*80}")
        print(f"RESULTS FOR: {construct_prefix.upper()} for {trait_of_interest}")
        print(f"{'='*80}\n")

        print(f"Mean Table for {construct_prefix}:")
        print(all_results[construct_prefix]["mean_table"])
        print("\n")

        print(f"Post-hoc Results for {construct_prefix} (High Level Only):")
        if "High" in all_results[construct_prefix]["posthoc_results"]:
            print(all_results[construct_prefix]["posthoc_results"]["High"])
        else:
            print("No High level data available")
        print("\n")




ANALYSIS BY TRAIT: NEUROTICISM


RESULTS FOR: QUALITY for neuroticism

Mean Table for quality:
  Trait Level  quality_openness  quality_conscientiousness  \
0         Low              4.25                       4.16   
1        High              4.20                       4.10   

   quality_extroversion  quality_agreeableness  quality_neuroticism  \
0                  4.30                   4.30                 4.26   
1                  4.35                   4.41                 4.45   

   quality_featurebased  Kruskal Statistic  p-value  
0                  4.34             10.290    0.067  
1                  4.38             16.314    0.006  


Post-hoc Results for quality (High Level Only):
                           quality_openness  quality_conscientiousness  \
quality_openness                        NaN                   1.000000   
quality_conscientiousness          1.000000                        NaN   
quality_extroversion               1.000000                   0.20095